In [1]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)


from sklearn.metrics import f1_score
import plotly.express as px
import seaborn as sns
import os, re
import numpy as np, pandas as pd
from scipy.stats import ttest_ind


# %cd /gdrive/My\ Drive/Colab\ Notebooks/
%cd /gdrive/MyDrive/TUM/ICAIL

# necessary imports


Mounted at /gdrive
/gdrive/MyDrive/TUM/ICAIL


In [2]:
df_merge = pd.read_csv('data/annotation/rave_disagreement.tsv',sep='\t')
df_merge.head()

,Unnamed: 0,case_id,known_o,article,pred_o,violated_x,vote,importance,unanimous,key_case,...,meta,interAnno_f1_0,interAnno_k_0,interAnno_f1_1,interAnno_k_1,allegations,violations,Unnamed: 3,Unnamed: 4,numPar
0,0,001-193590,0,3,1,1,0,2,1,0,...,1,0.390071,0.182199,0.402344,0.295147,3,3,NaN,NaN,49
1,1,001-193614,0,5,1,1,0,2,1,0,...,1,0.312343,0.215274,0.331950,0.275035,"5,6,8","5,8",NaN,NaN,26
2,2,001-193614,0,6,0,0,0,2,1,0,...,0,0.000000,-0.010898,0.000000,-0.005448,"5,6,8","5,8",NaN,NaN,26
3,3,001-193614,0,8,1,1,0,2,1,0,...,1,0.419753,0.282131,0.431487,0.354732,"5,6,8","5,8",NaN,NaN,26
4,4,001-194439,0,2,1,1,0,2,1,0,...,1,0.401940,0.217048,0.404531,0.313159,"2, 3, 6, 14, P1-1",2,NaN,NaN,71


In [5]:
##  preprocess proxy variables

#JudgeSplit:
''' (If the case decision is not a unanimous vote > It is a split vote)'''
df_merge['JudgeSplit'] = (df_merge['unanimous'] != 1).astype(int)

#Keycase:
''' whether the case is listed as keycase by ECtHR'''
df_merge['Keycase'] = (df_merge['key_case'] == 1).astype(int)

#OmitAlleg:
'''Subsidiary allegations absent in the original HUDOC metadata'''
df_merge['OmitAlleg'] = (df_merge['meta'] == 1).astype(int)

#DiffPred
'''Difference in Outcome Prediction'''
df_merge['DiffPred'] = (df_merge['pred_i'] != df_merge['pred_o']).astype(int)

#DiffFam
'''Difference in Case Familiarity'''
df_merge['DiffFam'] = (df_merge['known_i'] != df_merge['known_o']).astype(int)

#NumPar
'''utilize number of paragraphs'''
# df_merge['numPar']







'utilize number of paragraphs'

In [6]:
## t-test


def significatn_test(df, feature):
  # feature: binary proxy variables of the possible sources of disagreeement


# group the DataFrame by the values in column feature
  grouped = df.groupby(feature)

  # # get the groups for 0 and 1
  group0 = grouped.get_group(0)["interAnno_k_1"]
  group1 = grouped.get_group(1)["interAnno_k_1"]

  # get the mean of each group
  mean_0 = round(group0.mean(),3)
  mean_1 = round(group1.mean(),3)

  # # perform t-test on the two groups
  t_statistic, p_value = ttest_ind(group0, group1, equal_var=True)
  print(f'p_value: {p_value}')
  print(f't_statistic: {t_statistic}')

  print(f'mean_0: {mean_0}')
  print(f'mean_1: {mean_1}')

  # print results
  if p_value < 0.05:
    if mean_0 > mean_1:
      print(f"The mean of {feature} 0 is significantly bigger than that of group 1.")
    else:
      print(f"The mean of {feature} 1 is significantly bigger than that of group 0.")
  else:
      print("There is no significant difference between the means of group 0 and group 1.")


In [8]:
# Main

'''statistical association between proxy variables and expert’s IAA score.'''
proxy_variables = ['JudgeSplit','Keycase','OmitAlleg','DiffPred','DiffFam']

for proxy in proxy_variables:
  significatn_test(df_merge, proxy)

p_value: 0.0028095172062429113
t_statistic: -3.095729823931687
mean_0: 0.218
mean_1: 0.344
The mean of JudgeSplit 1 is significantly bigger than that of group 0.
p_value: 0.8555247997504345
t_statistic: -0.18273748189384495
mean_0: 0.242
mean_1: 0.249
There is no significant difference between the means of group 0 and group 1.
p_value: 8.500157514472284e-06
t_statistic: -4.800667416613973
mean_0: 0.086
mean_1: 0.278
The mean of OmitAlleg 1 is significantly bigger than that of group 0.
p_value: 0.03883616037592681
t_statistic: -2.104942958915564
mean_0: 0.225
mean_1: 0.315
The mean of DiffPred 1 is significantly bigger than that of group 0.
p_value: 0.6508758702443435
t_statistic: -0.4544715907860136
mean_0: 0.24
mean_1: 0.26
There is no significant difference between the means of group 0 and group 1.
